In [1]:
import pandas as pd

path = "data/mapdataall.csv"

fire_data = pd.read_csv(path)

fire_data_columns = ["incident_name", "incident_id", "incident_county", 
                     "incident_longitude", "incident_latitude", "incident_dateonly_created", 
                     "incident_dateonly_extinguished"]

fire_data_filtered = fire_data[fire_data_columns]
fire_data_filtered


# Columns to include:
# incident_name  
# incident_id
# incident_county 
# incident_acres_burned
# incident_dateonly_extinguished
# incident_dateonly_created
# incident_longitude
# incident_latitude

,incident_name,incident_id,incident_county,incident_longitude,incident_latitude,incident_dateonly_created,incident_dateonly_extinguished
0,Bridge Fire,2ca11d45-8139-4c16-8af0-880d99b21e82,Shasta,-122.309000,40.774000,2017-10-31,2018-01-09
1,Pala Fire,8f61f461-552d-4538-b186-35ab030da416,San Diego,1.000000,1.000000,2009-05-24,2009-05-25
2,River Fire,094719ba-a47b-4abb-9ec5-a506b2b9fd23,Inyo,-118.016510,36.602575,2013-02-24,2013-02-28
3,Fawnskin Fire,58f89ff8-bd3e-4355-b1c0-8fa05c747d3f,San Bernardino,-116.941311,34.288877,2013-04-20,2013-04-22
4,Gold Fire,357ffc13-bef9-48eb-810f-c5de851972eb,Madera,-119.635004,37.116295,2013-04-30,2013-05-01
...,...,...,...,...,...,...,...
2057,Irie Fire,cad849b4-be9c-4a47-889d-a1c1bcf2c085,Tulare,-118.752908,36.007323,2022-09-21,NaN
2058,Manzanita Fire,70bfeb43-3af8-4aa8-847c-07f0869d6f6d,San Bernardino,-117.152900,34.177600,2022-09-23,NaN
2059,97 Fire,a5373b17-81bc-4a24-a844-148cb5de6b10,Siskiyou,-122.341400,41.498700,2022-09-27,NaN
2060,Howard Fire,a139a767-908f-4275-9314-f3fb47bd9155,Ventura,-119.167059,34.560597,2022-10-08,NaN


In [2]:
fire_data_filtered['incident_dateonly_created'] = pd.to_datetime(fire_data_filtered['incident_dateonly_created'], errors ='ignore',
                                   yearfirst = True)
fire_data_filtered['incident_dateonly_extinguished'] = pd.to_datetime(fire_data_filtered['incident_dateonly_extinguished'], errors ='ignore',
                                   yearfirst = True)

fire_data_filtered

C:\Users\kylem\AppData\Local\Temp\ipykernel_23872\3043952833.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fire_data_filtered['incident_dateonly_created'] = pd.to_datetime(fire_data_filtered['incident_dateonly_created'], errors ='ignore',
C:\Users\kylem\AppData\Local\Temp\ipykernel_23872\3043952833.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fire_data_filtered['incident_dateonly_extinguished'] = pd.to_datetime(fire_data_filtered['incident_dateonly_extinguished'], errors ='ignore',


,incident_name,incident_id,incident_county,incident_longitude,incident_latitude,incident_dateonly_created,incident_dateonly_extinguished
0,Bridge Fire,2ca11d45-8139-4c16-8af0-880d99b21e82,Shasta,-122.309000,40.774000,2017-10-31,2018-01-09
1,Pala Fire,8f61f461-552d-4538-b186-35ab030da416,San Diego,1.000000,1.000000,2009-05-24,2009-05-25
2,River Fire,094719ba-a47b-4abb-9ec5-a506b2b9fd23,Inyo,-118.016510,36.602575,2013-02-24,2013-02-28
3,Fawnskin Fire,58f89ff8-bd3e-4355-b1c0-8fa05c747d3f,San Bernardino,-116.941311,34.288877,2013-04-20,2013-04-22
4,Gold Fire,357ffc13-bef9-48eb-810f-c5de851972eb,Madera,-119.635004,37.116295,2013-04-30,2013-05-01
...,...,...,...,...,...,...,...
2057,Irie Fire,cad849b4-be9c-4a47-889d-a1c1bcf2c085,Tulare,-118.752908,36.007323,2022-09-21,NaT
2058,Manzanita Fire,70bfeb43-3af8-4aa8-847c-07f0869d6f6d,San Bernardino,-117.152900,34.177600,2022-09-23,NaT
2059,97 Fire,a5373b17-81bc-4a24-a844-148cb5de6b10,Siskiyou,-122.341400,41.498700,2022-09-27,NaT
2060,Howard Fire,a139a767-908f-4275-9314-f3fb47bd9155,Ventura,-119.167059,34.560597,2022-10-08,NaT


In [3]:
fire_data_filtered = fire_data_filtered.sort_values(['incident_dateonly_created'],ascending = True)
start_date = pd.to_datetime('2013-1-1')
end_date = pd.to_datetime('2023-1-1')

In [4]:
fire_data_filtered = fire_data_filtered.loc[(fire_data_filtered['incident_dateonly_created'] > start_date)]
fire_data_filtered['year'] = fire_data_filtered['incident_dateonly_created'].dt.year
fire_data_filtered

,incident_name,incident_id,incident_county,incident_longitude,incident_latitude,incident_dateonly_created,incident_dateonly_extinguished,year
2,River Fire,094719ba-a47b-4abb-9ec5-a506b2b9fd23,Inyo,-118.016510,36.602575,2013-02-24,2013-02-28,2013
3,Fawnskin Fire,58f89ff8-bd3e-4355-b1c0-8fa05c747d3f,San Bernardino,-116.941311,34.288877,2013-04-20,2013-04-22,2013
4,Gold Fire,357ffc13-bef9-48eb-810f-c5de851972eb,Madera,-119.635004,37.116295,2013-04-30,2013-05-01,2013
6,Silverado Fire,d8d1c943-7b5d-4885-9043-f41075d6eded,Napa,-122.350844,38.441792,2013-04-30,2013-05-01,2013
5,Panther Fire,53122f0f-fefc-4dbf-b2d8-566b42ced66d,Tehama,-121.595555,40.190062,2013-05-01,2013-05-09,2013
...,...,...,...,...,...,...,...,...
2057,Irie Fire,cad849b4-be9c-4a47-889d-a1c1bcf2c085,Tulare,-118.752908,36.007323,2022-09-21,NaT,2022
2058,Manzanita Fire,70bfeb43-3af8-4aa8-847c-07f0869d6f6d,San Bernardino,-117.152900,34.177600,2022-09-23,NaT,2022
2059,97 Fire,a5373b17-81bc-4a24-a844-148cb5de6b10,Siskiyou,-122.341400,41.498700,2022-09-27,NaT,2022
2060,Howard Fire,a139a767-908f-4275-9314-f3fb47bd9155,Ventura,-119.167059,34.560597,2022-10-08,NaT,2022


In [5]:
fire_data_bins = [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
firedate_lables = [2013,2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]

In [6]:
fire_data_filtered['year']= pd.cut(fire_data_filtered.year,fire_data_bins,labels=firedate_lables)
fire_data_grouped = fire_data_filtered.groupby('year')


,incident_name,incident_id,incident_county,incident_longitude,incident_latitude,incident_dateonly_created,incident_dateonly_extinguished,year
2,River Fire,094719ba-a47b-4abb-9ec5-a506b2b9fd23,Inyo,-118.016510,36.602575,2013-02-24,2013-02-28,2013
3,Fawnskin Fire,58f89ff8-bd3e-4355-b1c0-8fa05c747d3f,San Bernardino,-116.941311,34.288877,2013-04-20,2013-04-22,2013
4,Gold Fire,357ffc13-bef9-48eb-810f-c5de851972eb,Madera,-119.635004,37.116295,2013-04-30,2013-05-01,2013
6,Silverado Fire,d8d1c943-7b5d-4885-9043-f41075d6eded,Napa,-122.350844,38.441792,2013-04-30,2013-05-01,2013
5,Panther Fire,53122f0f-fefc-4dbf-b2d8-566b42ced66d,Tehama,-121.595555,40.190062,2013-05-01,2013-05-09,2013
143,Campbell Fire,353e7df7-440c-4c41-a27c-b35bb1e2831c,Tehama,-121.790183,40.134597,2014-01-02,2014-01-10,2014
145,Red Fire,6d75c6a4-12cb-4f18-8951-f2737e980916,Humboldt,-123.773779,40.884415,2014-01-04,2014-01-08,2014
144,Colby Fire,1fe0ea01-23e8-47a1-8fbd-ced6cf655e4d,Los Angeles,-117.881600,34.170300,2014-01-16,2014-01-27,2014
146,Lake Fire,d529636e-33d0-484b-9c6f-093ba33736b7,Fresno,-119.341500,36.826600,2014-01-25,2014-01-26,2014
147,Gun Fire,a67fe9bc-b8ec-4ed1-a2e8-06623aab7039,San Diego,-116.490000,32.810000,2014-03-08,2014-03-11,2014
